## Benchmarks de AIONER

En este notebook vamos a realizar los benchmarks con AIONER, para ello comenzaremos partiendo del repositorio original de [AIONER](https://github.com/ncbi/AIONER) al cual se la realizarn varias modificaciones, las cuales se encuentran en este [repositorio forkeado](https://github.com/manumartinm/AIONER)

A continuación cargaremos los datos en formato Pubtator y por último obtendremos los resultados juntos con las métricas de evaluación, las cuales guardaremos en un JSON.

### Librerías

In [2]:
from datasets_utils import datasets_map
from ner_benchmark_data_wrangler import NERBenchmarkDataWrangler
from utils import create_directories, save_to_file
import subprocess

### Carga de datos

Vamos a cargar los datos tanto de entrenamiento, validación y test en formato Pubtator en el directorio `custom_datasets`, también obtendremos el vocab y lo guardaremos en el directorio `custom_vocab`.

In [ ]:
initial_dirs = ['./custom_datasets', './custom_vocab', './metrics']

create_directories(initial_dirs)

for dataset in datasets_map.keys():
    wrangler = NERBenchmarkDataWrangler(dataset, datasets_map)
    
    train_df = wrangler.get_format_data('train')
    valid_df = wrangler.get_format_data('valid')
    test_df = wrangler.get_format_data('test')
    
    train_pubtator = wrangler.to_pubtator('train')
    test_pubtator = wrangler.to_pubtator('test')
    valid_pubtator = wrangler.to_pubtator('valid')

    join_texts = train_pubtator + valid_pubtator + test_pubtator
    
    labels = list(set(train_df['labels'].unique().tolist() +
                      valid_df['labels'].unique().tolist() +
                      test_df['labels'].unique().tolist()))
    
    dataset_dirs = [f'./custom_datasets/{dataset}', f'./custom_vocab/{dataset}']

    create_directories(dataset_dirs)
    
    save_to_file(f'./custom_datasets/{dataset}/texts.txt', '\n\n'.join(join_texts))
    save_to_file(f'./custom_vocab/{dataset}.txt', labels)


### Obtención de resultados y métricas

Por último, obtendremos los resultados usando el script que nos proporciona AIONER, el cual es `AIONER_Run.py` y calcularemos las métricas de evaluación con el script `Test_Evaluation.py`, las métricas se guardaran en el directorio `custom_metrics`.

In [ ]:
for dataset in datasets_map.keys():
    subprocess.run(
        f'python ../AIONER/src/AIONER_Run.py -i ./custom_datasets/{dataset}/ -m ../AIONER/pretrained_models/AIONER/Bioformer-softmax-AIONER.h5 -v ./custom_vocab/{dataset}.txt -e ALL -o output/{dataset}/',
        shell=True
    )

    subprocess.run(
        f'python ../AIONER/src/Test_Evaluation.py --gold ./custom_datasets/{dataset}/test.txt --pred output/{dataset}/test.txt --output./metrics/{dataset}.json',
        shell=True
    )